Timer start

In [1]:
from time import perf_counter

start_script_time = perf_counter()

# Init Block

In [2]:
# imports
from datetime import date, timedelta, datetime
from pathlib import Path
import re
import logging
from dotenv import dotenv_values
import requests
import ast

import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values, DictCursor

import gspread
from google.oauth2.service_account import Credentials

import smtplib
import ssl
from email.message import EmailMessage

ENV_FILE=dotenv_values('.env')

# -------------Bamboo PostgreSQL DB config-------------
BAMBOO_DB_DSN = ENV_FILE.get('DB_DSN_bamboo')

# -------------Google Sheets config-------------
GS_CREDS_FILE = Path('bamboo_service_account.json')  # path to json google key

DASHBOARD_SPREADSHEET_ID = "1U5ApJgj2_4CWYd_7nV1_LUvUL1aKmU8dB96OlLSd170"      # id from URL
DASHBOARD_WORKSHEET_NAME = "Лист1"                         # list name
DASHBOARD_SCOPES = ["https://www.googleapis.com/auth/spreadsheets"] # rights request

# -------------API and download period config-------------

# today time period
DOWNLOAD_TIME_START = f"{date.today()} 00:00:00.000000"
DOWNLOAD_TIME_END = f"{date.today()} 23:59:59.999999"

#fixed time period 
# DOWNLOAD_TIME_START = '2025-01-06 12:46:47.860798'
# DOWNLOAD_TIME_END = '2026-01-06 13:50:47.860798'

ITRESUME_URL = "https://b2b.itresume.ru/api/statistics"
ITRESUME_SKILLFACTORY_PARAMS={
'client': 'Skillfactory',
'client_key':ENV_FILE.get('CLIENT_KEY_Skillfactory'),
'start':DOWNLOAD_TIME_START,
'end':DOWNLOAD_TIME_END
}

# -------------GMAIL config-------------
BARROCO_SMTP = 'smtp.gmail.com'
BARROCO_EMAIL_PORT= 465
BARROCO_ACCOUNT_PASSWORD = ENV_FILE.get('GOOGLE_APP_PASS')
BARROCO_EMAIL = 'unpocobarroco@gmail.com'
BARROCO_EMAIL_NAME = 'Bamboo Project'

# DEFINITIONS

### Classes

- **DB_Ops**: cummunication with DB
- **Mail_Server_SSL**: cummunication with Mail-sevice Server
    - SSL connection with SMTP
    - Use `with...as` to ensure SMTP-session is closed

In [3]:
class DbOps:
    """ Class for more simple communication with database according this ETL aims
    """
    
    def __init__(self, db_dsn:str ):
        self.dsn=db_dsn
        
    def make_table(self, schema, table, make_table_query): 
        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor() as cur:
                
                ready_query=sql.SQL(make_table_query).format(sql.Identifier(schema, table))
                cur.execute(ready_query)
    
    @staticmethod
    def make_sql_columns(columns):
        return sql.SQL(', ').join(sql.Identifier(col) for col in columns)
    
    
    def insert_unique_data_to_table(self, schema, table, columns, data):
        
        if not data:
            return 0
        
        needed_length=len(columns)
        for i, row in enumerate(data):
            if len(row)!=needed_length:
                raise ValueError(f"{i} row length is not the same as columns count in DB table ({needed_length})")
        
        columns_sql=DbOps.make_sql_columns(columns)
        
        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor() as cur:
        
                query=sql.SQL(
                """
                INSERT INTO {} ({})
                VALUES %s
                ON CONFLICT ON CONSTRAINT unique_check DO NOTHING
                RETURNING 1
                """).format(sql.Identifier(schema, table), columns_sql)
                
                returned = execute_values(cur, query, data, page_size=1000, fetch=True)
                return len(returned)
                
            
    def query_fetchall_dict(self, query):
        with psycopg2.connect(self.dsn) as conn:
            with conn.cursor(cursor_factory=DictCursor) as cur:
                
                cur.execute(query)
                return cur.fetchall()


class MailServerSSL:
    
    context = ssl.create_default_context()
    
    def __init__(self, smtp_server:str, ssl_port:int, account_password:str, sender_email:str, sender_name:str):
        self.account_password=account_password
        self.smtp_server = smtp_server
        self.ssl_port = ssl_port
        self.sender_email = sender_email
        self.sender_name=sender_name
        
    def send_message(self, recipient_email, subject, message):
        msg = EmailMessage()
        
        msg['Subject'] = subject
        msg['From'] = f"{self.sender_name} <{self.sender_email}>"
        msg['To'] = recipient_email
        msg.set_content(message)
        
        
        with smtplib.SMTP_SSL(self.smtp_server, self.ssl_port, context=MailServerSSL.context) as server:
            server.login(self.sender_email, self.account_password)
            server.send_message(msg)
            
        return msg
    

class GoogleSpreadsheet:
    
    def __init__(self, creds_file: str, scopes: list[str], spreadsheet_id: str):
        
        self.creds = Credentials.from_service_account_file(creds_file, scopes=scopes)
        self.gc = gspread.authorize(self.creds)         # client
        self.sh = self.gc.open_by_key(spreadsheet_id)   # spreadsheet
        self.ws = None                                  # worksheet

    def select_worksheet(self, worksheet_name: str):
        
        if self.ws and self.ws.title == worksheet_name:
            return self.ws
        
        self.ws = self.sh.worksheet(worksheet_name)
        return self.ws

###  Helper functions

In [4]:
def cleanup_logs(log_dir, keep_limit_days):

    today_date=date.today()
    keep_limit_date=today_date-timedelta(days=keep_limit_days)

    log_dir.mkdir(parents=True, exist_ok=True)

    for p in log_dir.iterdir():
        
        if not p.is_file():
            continue
        
        filename=p.name
        match_date_in_name=re.search(r"\d{4}-\d{2}-\d{2}",filename)
        
        if not match_date_in_name:
            continue
        iso_date_in_name=date.fromisoformat(match_date_in_name.group())
        
        if iso_date_in_name > keep_limit_date:
            continue
        
        p.unlink()
   
        
def get_periods(total_seconds):

    days, reminder = divmod(total_seconds, 86400)
    hours, reminder = divmod(reminder, 3600)
    mins, secs = divmod(reminder, 60)

    return {'d': days, 'h': hours, 'm': mins, 's': secs}
        

def string_to_dict(string):
    
    try:
        return ast.literal_eval(string)
        
    except Exception:
        logging.warning(f"Not able to convert passback_params to dict format")
        raise ValueError("String-to-dict convertation Error")
        

def convert_to_datetime(string):
    
    try:
        return datetime.fromisoformat(string)
    
    except Exception:
        logging.warning(f"Not able to convert {string} to ISO-format.")
        raise ValueError("Date Convertation Error")


def validate_return_value(value, accepted_vals: tuple | None = None, class_or_tuple = None):
    
    if accepted_vals is not None and (value not in accepted_vals):
        logging.warning(f"Value Validation Error: {value} is not in {accepted_vals}")
        raise ValueError ("Value Validation Error")
    
    if class_or_tuple is not None and not isinstance(value, class_or_tuple):
        logging.warning(f"Type Validation Error: {value} doesn`t fit {class_or_tuple}")
        raise ValueError("Type Validation Error")
    
    return value


def hook(obj):
    
    try:
        # extract the vals according the correct structure as it is
        user_id = obj['lti_user_id']
        oauth_consumer_key = string_to_dict(obj['passback_params']).get('oauth_consumer_key', None)
        lis_result_sourcedid = string_to_dict(obj['passback_params']).get('lis_result_sourcedid', None)
        lis_outcome_service_url = string_to_dict(obj['passback_params']).get('lis_outcome_service_url', None)
        is_correct = obj['is_correct']
        attempt_type = obj['attempt_type']
        created_at = obj['created_at']
        
        # validate and put to the structure
        plain_dict = {
            'user_id':validate_return_value(user_id, class_or_tuple=str), # строковый айди пользователя
            'oauth_consumer_key': validate_return_value(oauth_consumer_key, class_or_tuple=(str, type(None))), # уникальный токен клиента
            'lis_result_sourcedid': validate_return_value(lis_result_sourcedid, class_or_tuple=(str, type(None))), # ссылка на блок, в котором находится задача в ЛМС
            'lis_outcome_service_url': validate_return_value(lis_outcome_service_url, class_or_tuple=(str, type(None))), # URL адрес в ЛМС, куда мы шлем оценку
            'is_correct': validate_return_value(is_correct, accepted_vals=(0, 1, None), class_or_tuple=(int, type(None))), # была ли попытка верной (null, если это run)
            'attempt_type': validate_return_value(attempt_type, accepted_vals=('run', 'submit'), class_or_tuple=str), # ран или сабмит
            'created_at': convert_to_datetime(created_at)  # дата и время попытки
            }
        
        return plain_dict
    
    except Exception:
        logging.warning(f"Above fail araised in: created_at {created_at}, user_id {user_id}")
        
        return

# Bootstrap
- Log directory manage
- Log files cleanup
- Logging settings: force=True - гарантия создания нового файла при смене даты

In [5]:
log_dir = Path("logs")
keep_limit_days=3

cleanup_logs(log_dir, keep_limit_days)

log_file = log_dir / f"logs ({date.today()}).txt"
logging.basicConfig(filename=log_file,level=logging.INFO, filemode='a', format="%(asctime)s %(name)s %(levelname)s: %(message)s", force=True)

# ETL

## Extracting API

### Downloading data by API
- using extra visual check (first_row_data) for further steps

In [6]:
logging.info("====================== New ETL started ======================")
logging.info("Start downloading data by API")

try:
    
    r=requests.get(ITRESUME_URL, params=ITRESUME_SKILLFACTORY_PARAMS, timeout=(10, 60))
    
    r.raise_for_status()
    first_row_data=r.json()
    rows_income_cnt=len(first_row_data)
    
    logging.info(f"Downloading data by API completed successfully. Code {r.status_code}. {rows_income_cnt} rows total")
    
except Exception:
    logging.exception("Invalid get-parameters or read timed out")
    raise


first_row_data

[{'lti_user_id': '3672b7392e3404e7cbd41ef3b5f9fef3',
  'passback_params': "{'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+DSPR-2.0+14JULY2021:lms.skillfactory.ru-c387b2c574f64e4bb049741fca4dfc8e:3672b7392e3404e7cbd41ef3b5f9fef3', 'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/xblock/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@lti+block@c387b2c574f64e4bb049741fca4dfc8e/handler_noauth/grade_handler'}",
  'is_correct': None,
  'attempt_type': 'run',
  'created_at': '2026-01-09 01:01:02.268413'},
 {'lti_user_id': 'd17982fdf1a034ef99c2e397f0666066',
  'passback_params': "{'oauth_consumer_key': '', 'lis_result_sourcedid': 'course-v1:SkillFactory+SQL2.0+31AUG2020:lms.skillfactory.ru-df97b0d979e9418198b060b73c51272c:d17982fdf1a034ef99c2e397f0666066', 'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+SQL2.0+31AUG2020/xblock/block-v1:SkillFactory+SQL2.0+31AUG2020+type@l

## Data transform

### Validate and transform of downloaded data

In [7]:
logging.info("----- Raw data transformation started -----")    

try:    
    
    user_solutions=r.json(object_hook=hook)
    logging.info("Transformation stage 1 (rows transformation and validation) completed")   
    
except Exception:
    logging.exception("Transformation stage 1 (rows transformation and validation) failed")
    raise

user_solutions

[{'user_id': '3672b7392e3404e7cbd41ef3b5f9fef3',
  'oauth_consumer_key': '',
  'lis_result_sourcedid': 'course-v1:SkillFactory+DSPR-2.0+14JULY2021:lms.skillfactory.ru-c387b2c574f64e4bb049741fca4dfc8e:3672b7392e3404e7cbd41ef3b5f9fef3',
  'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/xblock/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@lti+block@c387b2c574f64e4bb049741fca4dfc8e/handler_noauth/grade_handler',
  'is_correct': None,
  'attempt_type': 'run',
  'created_at': datetime.datetime(2026, 1, 9, 1, 1, 2, 268413)},
 {'user_id': 'd17982fdf1a034ef99c2e397f0666066',
  'oauth_consumer_key': '',
  'lis_result_sourcedid': 'course-v1:SkillFactory+SQL2.0+31AUG2020:lms.skillfactory.ru-df97b0d979e9418198b060b73c51272c:d17982fdf1a034ef99c2e397f0666066',
  'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+SQL2.0+31AUG2020/xblock/block-v1:SkillFactory+SQL2.0+31AUG2020+type@lti+block@df97b0d979e941819

### Get rid of the skipped rows ("None`s") in the list of user solutions 
- clearing nearly in-place to safe RAM

In [8]:
try:

    user_solutions[:]=[x for x in user_solutions if x is not None]
    logging.info("Transformation stage 2 (non-working rows clearing) completed")

    rows_transformed_cnt=len(user_solutions)
    
    logging.info(f"All stages of raw data transformation completed: {rows_transformed_cnt} rows of {rows_income_cnt} done")

except Exception:
    logging.exception("Transformation stage 2 (non-working rows clearing) failed")
    raise


user_solutions

[{'user_id': '3672b7392e3404e7cbd41ef3b5f9fef3',
  'oauth_consumer_key': '',
  'lis_result_sourcedid': 'course-v1:SkillFactory+DSPR-2.0+14JULY2021:lms.skillfactory.ru-c387b2c574f64e4bb049741fca4dfc8e:3672b7392e3404e7cbd41ef3b5f9fef3',
  'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DSPR-2.0+14JULY2021/xblock/block-v1:SkillFactory+DSPR-2.0+14JULY2021+type@lti+block@c387b2c574f64e4bb049741fca4dfc8e/handler_noauth/grade_handler',
  'is_correct': None,
  'attempt_type': 'run',
  'created_at': datetime.datetime(2026, 1, 9, 1, 1, 2, 268413)},
 {'user_id': 'd17982fdf1a034ef99c2e397f0666066',
  'oauth_consumer_key': '',
  'lis_result_sourcedid': 'course-v1:SkillFactory+SQL2.0+31AUG2020:lms.skillfactory.ru-df97b0d979e9418198b060b73c51272c:d17982fdf1a034ef99c2e397f0666066',
  'lis_outcome_service_url': 'https://lms.skillfactory.ru/courses/course-v1:SkillFactory+SQL2.0+31AUG2020/xblock/block-v1:SkillFactory+SQL2.0+31AUG2020+type@lti+block@df97b0d979e941819

## Load to Database

### Creating a table in DB if doesn`t exist + put the info into table

In [9]:
logging.info("----- Start communication with Database -----")

try:

    bamboo=DbOps(BAMBOO_DB_DSN)

    table_schema ='public'
    table_name = 'solutions'
    target_in_log=f"{table_schema}.{table_name}"
    
    make_table_query="""
    CREATE TABLE IF NOT EXISTS {} (
        id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY
        , user_id TEXT NOT NULL
        , oauth_consumer_key TEXT NOT NULL
        , lis_result_sourcedid TEXT NOT NULL
        , lis_outcome_service_url TEXT NOT NULL
        , is_correct INT CHECK (is_correct in (0, 1) OR is_correct is NULL)
        , attempt_type TEXT CHECK (attempt_type in ('run', 'submit')) NOT NULL
        , created_at TIMESTAMP NOT NULL
        , CONSTRAINT unique_check UNIQUE (user_id, created_at)
    )
    """
    
    bamboo.make_table(table_schema, table_name, make_table_query)
    logging.info(f"[{target_in_log}] Checking/creating table completed successfully")
    
    columns_for_inserting=('user_id', 'oauth_consumer_key', 'lis_result_sourcedid', 'lis_outcome_service_url', 'is_correct', 'attempt_type', 'created_at')
      
    tuples_to_insert=[(
        row["user_id"],
        row["oauth_consumer_key"],
        row["lis_result_sourcedid"],
        row["lis_outcome_service_url"],
        row["is_correct"],
        row["attempt_type"],
        row["created_at"]
    ) for row in user_solutions]
    rows_prepared_cnt = len(tuples_to_insert)
    logging.info(f"[{target_in_log}] Insert-tuples list preparing completed successfully") 
    
    inserted_rows_cnt = bamboo.insert_unique_data_to_table(table_schema, table_name, columns_for_inserting, tuples_to_insert)
    logging.info(f"[{target_in_log}] Inserting data to DB-table completed. {inserted_rows_cnt} new rows of {rows_prepared_cnt} prepared were inserted")
    
    logging.info("Communication with Database completed successfully") 

except Exception:
    logging.exception("Communication with Database failed")
    raise

## SQL Analytics

### Prepare aggregated data dict, using DB
- Always aggregate all data because:
    1) the previous data can be added the following day because of the timezone
    2) worksheet can be changed by everyone, BD - can`t, so need to update correct info

In [ ]:
logging.info("----- Direct database analytics starts -----")

try:    

	db_dic_list= bamboo.query_fetchall_dict(
	""" 
	WITH 
		first_seen AS (
			SELECT 
				min(date(created_at)) AS study_date
				, user_id
			FROM public.solutions
			GROUP BY user_id
		),
		
		new_users AS (
			SELECT
				study_date
				, count(user_id) AS new_users_cnt
			FROM first_seen
			GROUP BY study_date
		),

		solutions_metrics AS (
			SELECT 
				date(created_at) AS study_date
				, count(*) AS attempts_per_day_total
				, count(*) FILTER(WHERE is_correct=1) AS correct_attempts_per_day
				, count(DISTINCT user_id) AS unique_users_per_day	
			FROM public.solutions
			GROUP BY study_date
			ORDER BY study_date
		)
			
	SELECT 
		to_char(sm.study_date, 'yyyy-mm-dd') AS study_date
		, attempts_per_day_total
		, correct_attempts_per_day
		, unique_users_per_day	
		, COALESCE(nu.new_users_cnt, 0) AS new_users_per_day
	FROM solutions_metrics sm
	LEFT JOIN new_users nu USING(study_date)
	ORDER BY study_date
	"""
	)
	
	fetch_rows_cnt = len(db_dic_list)
	logging.info(f"Aggregated report ({fetch_rows_cnt} rows) for Google Spreadsheets Dashboard was successfully fetched by using SQL query to DB")

except Exception:
    logging.exception("SQL query construction failed")
    raise


db_dic_list

[['2025-12-25', 119, 54, 10, 10],
 ['2026-01-05', 546, 169, 36, 36],
 ['2026-01-06', 987, 264, 57, 41],
 ['2026-01-07', 911, 199, 44, 22],
 ['2026-01-08', 1446, 353, 73, 41],
 ['2026-01-09', 83, 19, 5, 0]]

## Communucation with Google Spreadsheet

### Sheet headers prepare directly in WS
- get existing Google ws headers, 
- check if headers are the same as in DB, add new headers from DB if needed (nothing deletes)

In [11]:
logging.info("----- Start communucation with Google Spreadsheet -----")

try:
     
    ws = GoogleSpreadsheet(GS_CREDS_FILE, DASHBOARD_SCOPES, DASHBOARD_SPREADSHEET_ID).select_worksheet(DASHBOARD_WORKSHEET_NAME)
    
    logging.info("Connecting to Google Spreadsheet is stable")

except Exception:
    logging.exception("Using of Google Sheets credentials failed")

try:
    
    if fetch_rows_cnt==0:
        logging.warning("No rows for dashboard. Skipping Worksheet update")
        
    else:
        ws_headers=ws.row_values(1)
        query_headers=db_dic_list[0].keys()

        add_headers=[header for header in query_headers if header not in ws_headers]
            
        if add_headers:
            ws_headers.extend(add_headers)
            ws.update(range_name='1:1', values=[ws_headers])
            
        logging.info("Worksheet headers preparing completed successfully")

except Exception:
    logging.exception("Headers preparing in Worksheet failed")
 
        
ws_headers

['study_date',
 'attempts_per_day_total',
 'correct_attempts_per_day',
 'unique_users_per_day',
 'new_users_per_day']

### Add aggregated data to WS in Google Sheets
- If date exists- row replace, if not - row append. 
- Can find the date-column dinamicaly. 
- The row structure automatically fits the Google ws neww structure.
- Lines qty doesn`t validate
- Columns width doesn`t change
- Columns with same name doesn`t validate - just change both

In [12]:
try:
    
    dates_in_ws=ws.col_values(ws_headers.index('study_date')+1)

    for dic in db_dic_list:
        row_to_add=[dic.get(key) for key in ws_headers]
        row_date=dic['study_date']
        
        if row_date in dates_in_ws:
            ws.update(range_name=f"A{dates_in_ws.index(row_date)+1}", values=[row_to_add], value_input_option="USER_ENTERED")
            
        else:
            ws.append_row(row_to_add, value_input_option="USER_ENTERED")
            
    logging.info("Aggregated analytics appending to Google Worksheet completed successfully")
    logging.info("Communucation with Google Spreadsheet completed successfully")

except Exception:
    logging.exception("Failed to append analytical report data to Google Worksheet")
    raise

## Communucation with SMTP Server

### Send digest message via email

In [13]:
logging.info("----- Start communucation with SMTP Server -----")

try:
        
    gmail = MailServerSSL(BARROCO_SMTP, BARROCO_EMAIL_PORT, BARROCO_ACCOUNT_PASSWORD, BARROCO_EMAIL, BARROCO_EMAIL_NAME)

    message=f"""
    Dear all,
    
    Thanks for following our team digest!
    
    Todays ETL script was runned successfully:
    Rows extracted by API: {rows_income_cnt}
    Rows validated: {rows_transformed_cnt}
    Rows prepared to insert to DB: {rows_prepared_cnt}
    New rows inserted to DB: {inserted_rows_cnt}
    
    The dashboard Google Sheet was also updated successfully.
    
    BR,
    Bamboo team
    """
    logging.info("Email digest message preparing completed successfully")
    
    msg = gmail.send_message('fluxor@atomicmail.io', "Work done", message)
    logging.info("Email digest message was sent successfully")
    
    logging.info("Communucation with SMTP Server completed successfully")

except Exception:
    logging.exception("Communucation with SMTP Server failed")
    raise    

    
print(msg)

Subject: Work done
From: Bamboo Project <unpocobarroco@gmail.com>
To: fluxor@atomicmail.io
Content-Type: text/plain; charset="utf-8"
Content-Transfer-Encoding: 7bit
MIME-Version: 1.0


    Dear all,

    Thanks for following our team digest!

    Todays ETL script was runned successfully:
    Rows extracted by API: 83
    Rows validated: 83
    Rows prepared to insert to DB: 83
    New rows inserted to DB: 4

    The dashboard Google Sheet was also updated successfully.

    BR,
    Bamboo team
    



Timer stop

In [14]:
logging.info("----- ETL script finished successfully -----")
logging.info(f"Rows extracted by API: {rows_income_cnt}")
logging.info(f"Rows validated: {rows_transformed_cnt} ")
logging.info(f"Rows prepared to insert to DB: {rows_prepared_cnt}")
logging.info(f"New rows inserted to DB: {inserted_rows_cnt} ")

extract_seconds = round((datetime.fromisoformat(DOWNLOAD_TIME_END) - datetime.fromisoformat(DOWNLOAD_TIME_START)).total_seconds())
extracted_periods = get_periods(extract_seconds)

e_seconds = f"{round(extracted_periods['s'], 2)} seconds"
e_minutes = '' if extracted_periods['m']==0 else f"{extracted_periods['m']} minutes, "
e_hours= '' if extracted_periods['h']==0 else f"{extracted_periods['h']} hours, "
e_days= '' if extracted_periods['d']==0 else f"{extracted_periods['d']} days, "
logging.info(f"API Extracted period: {e_days}{e_hours}{e_minutes}{e_seconds}")


finish_script_seconds = perf_counter() - start_script_time
finish_periods = get_periods(finish_script_seconds)

f_seconds = f"{round(finish_periods['s'], 2)} seconds"
f_minutes = '' if finish_periods['m']==0 else f"{finish_periods['m']} minutes, "
f_hours= '' if finish_periods['h']==0 else f"{finish_periods['h']} hours, "
f_days= '' if finish_periods['d']==0 else f"{finish_periods['d']} days, "
logging.info(f"Script total run time: {f_days}{f_hours}{f_minutes}{f_seconds}")